In [99]:
import numpy as np
import copy
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit import BasicAer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_state_qsphere
from scipy.optimize import minimize
import seaborn

In [100]:
PROFITS = [3, 1, 2, 6, 1, 4, 1]
WEIGHTS = [2, 1, 1, 1, 3, 2, 1]
N_QUBITS = len(PROFITS)
THETA_S = [0.5 for _ in range(N_QUBITS)]
MAX_WEIGHT = 7
INIT_TYPE = "smoothened"
SOLVING_TYPE = "Copula"

In [112]:
class Subcircuit:

    def __init__(self, profits, weights, probs, thetas, gamma=None, beta=None):
        self.profits = profits
        self.weights = weights
        self.probs = probs
        self.thetas = thetas
        self.n_qubits = len(profits)
        self.gamma = gamma
        self.beta = beta
        self.qubits = QuantumRegister(self.n_qubits)
        self.circuit = QuantumCircuit(self.qubits)

    def get_circuit(self):
        return self.circuit
        

In [113]:
class InitializationGates(Subcircuit):

    def __init__(self, profits, weights, probs, thetas):
        super().__init__(profits, weights, probs, thetas)

        # Qubits werden in den rotierten Eigenzuständen gemäß den WSKs p_i initiiert
        for i in range(self.n_qubits):
            init = 2 * np.arccos(np.sqrt(1 - probs[i]))
            self.circuit.u(init, 0, 0, self.qubits[i])
        self.circuit.barrier()

    def get_circuit(self):
        return self.circuit

In [114]:
class CostGate(Subcircuit):

    def __init__(self, profits, weights, probs, thetas, gamma, beta):
        super().__init__(profits, weights, probs, thetas, gamma, beta)

        # auf jeden Qubit wird der unitäre Operator korrespondierend zu den Kosten des aktuellen Zustands angewandt
        for i in range(self.n_qubits):
            self.circuit.rz(self.gamma * profits[i] * 2, self.qubits[i])
        self.circuit.barrier()

In [127]:
class CopulaMixingGate(Subcircuit):
    
    def __init__(self, profits, weights, probs, thetas, gamma, beta):
        super().__init__(profits, weights, probs, thetas, gamma, beta)

        # Liste von interagierenden bzw. zu verschränkenden Qubits erstellen
        rng = list(range(self.n_qubits))
        indices = list(zip(rng[::2], rng[1::2]))
        rng.append(rng.pop(0))
        indices += list(zip(rng[::2], rng[1::2]))

        for (i, j) in indices:
            p1 = self.probs[i]
            p2 = self.probs[j]
            theta = self.thetas[i]

            # Wahrscheinlichkeiten bestimmen
            p21 = p2 + theta * p2 * (1 - p1) * (1 - p2)
            p2_1 = p2 - theta * p1 * p2 * (1 - p2)
            phi_p1 = 2 * np.arcsin(np.sqrt(p1))
            phi_p21 = 2 * np.arcsin(np.sqrt(p21))
            phi_p2_1 = 2 * np.arcsin(np.sqrt(p2_1))
            # Rotations-Circuit definieren
            q_s = QuantumRegister(2)
            R_p12 = QuantumCircuit(q_s)
            R_p12.ry(phi_p1, 0)
            # hier evtl cry-gate verwenden?
            R_p12.cu(phi_p21, 0, 0, 0, q_s[0], q_s[1])
            R_p12.x(0)
            # hier ebenso
            R_p12.cu(phi_p2_1, 0, 0, 0, q_s[0], q_s[1])
            R_p12.x(0)
            # Rotations-Circuits hintereinanderschalten
            self.circuit.barrier()
            self.circuit = self.circuit.compose(R_p12.inverse(), qubits=[i, j])
            self.circuit.rz(2 * self.beta, i)
            self.circuit.rz(2 * self.gamma, j)
            self.circuit = self.circuit.compose(R_p12, qubits=[i, j])
            self.circuit.barrier()

In [134]:
class HourglassMixingGate(Subcircuit):
    def __init__(self, profits, weights, probs, thetas, gamma, beta):
        super().__init__(profits, weights, probs, thetas, gamma, beta)

        for i in range(self.n_qubits):
            phi = 2 * np.arcsin(np.sqrt(self.probs[i]))
            self.circuit.ry(phi, self.qubits[i]).inverse()
            self.circuit.rz(2 * self.beta, self.qubits[i])
            self.circuit.ry(phi, self.qubits[i])

In [135]:
class TotalCircuit(object):

    # params: Parameter beta_i, gamma_i im Format[betas] + [gammas]
    def __init__(self, profits, weights, probs, thetas, params, type):
        self.profits = profits
        self.weights = weights
        self.probs = probs
        self.thetas = thetas
        self.p = len(params) // 2
        self.betas = params[:self.p]
        self.gammas = params[self.p:]
        self.n_qubits = len(profits)
        self.qubits = QuantumRegister(self.n_qubits)
        self.bits = ClassicalRegister(self.n_qubits)
        self.circuit = QuantumCircuit(self.qubits, self.bits)
        args = [profits, weights, probs, thetas]
        self.init_circuit = InitializationGates(*args)
        self.cost_gates = [CostGate(*args + [self.gammas[i], self.betas[i]]).circuit for i in range(len(self.gammas))]
        if type == "Copula":
            self.mixing_gates = [CopulaMixingGate(*args + [self.gammas[i], self.betas[i]]).circuit for i in range(len(self.gammas))]
        else:
            self.mixing_gates = [HourglassMixingGate(*args + [self.gammas[i], self.betas[i]]).circuit for i in range(len(self.gammas))]
        self.rng = list(range(self.n_qubits))
        
        # Initiierungsgates anwenden
        self.circuit = self.circuit.compose(self.init_circuit.get_circuit(), qubits=self.rng)

        # in p Durchgängen werden jeweils Cost- und Mixing-Unitary angewandt:
        for i in range(self.p):
            self.circuit = self.circuit.compose(self.cost_gates[i], qubits=self.rng)
            self.circuit = self.circuit.compose(self.mixing_gates[i], qubits=self.rng)

In [136]:
class Preprocessor(object):

    def __init__(self, profits, weights, k):
        self.profits = profits
        self.weights, self.max_weight = weights
        self.n_qubits = len(self.profits)
        self.k = k
    
    def lazy_greedy(self):
        items = [[i, self.profits[i], self.weights[i], self.profits[i] / self.weights[i], 0] for i in range(self.n_qubits)]
        items.sort(key=(lambda x: x[3]), reverse=True)
        total_weight = 0
        r_stop = None
        for item in items:
            if total_weight + item[2] <= self.max_weight:
                total_weight += item[2]
                item[-1] = 1
            else:
                r_stop = item[3] if r_stop is None else r_stop
        items.sort(key=(lambda x: x[0]))
        return [i[-1] for i in items], r_stop

    def bitstring_to_probs(self, type):
        if type == "constant":
            return np.array([self.max_weight / sum(self.weights) for _ in range(self.n_qubits)])
        elif type == "lazy greedy":
            lg = self.lazy_greedy()
            r_stop = lg[1]
            return np.array((np.array(self.profits) / np.array(self.weights)) > r_stop).astype(int)
        elif type == "smoothened":
            lg = self.lazy_greedy()
            r_stop = lg[1]
            C = sum(self.weights) / self.max_weight - 1
            func = np.vectorize(lambda x: 1 / (1 + C * np.exp(-1 * self.k * (x - r_stop))))
            return func(np.array(self.profits) / np.array(self.weights))
        else:
            print("Invalider Initialisierungstyp")
            return

In [194]:
class Solver(object):

    def __init__(self, profits, weights, thetas, solving_type, init_type, k):
        self.profits = profits
        self.thetas = thetas
        self.weights, self.max_weight = weights
        self.circuit = None
        self.params_set = False
        self.solving_type = solving_type
        self.probs = Preprocessor(profits, weights, k).bitstring_to_probs(init_type)


    def set_params(self, params):
        self.circuit = TotalCircuit(self.profits, self.weights, self.probs, self.thetas, params, self.solving_type)
        self.params_set = True
        

    def process_outcome(self):
        if not self.params_set:
            print("Parameter nicht initialisiert")
            return 
        copy_circuit = self.circuit.circuit.copy()
        copy_circuit.measure_all(add_bits=False)
        backend = BasicAer.get_backend("qasm_simulator")
        job = backend.run(transpile(copy_circuit, backend), shots=max(10, self.circuit.n_qubits))
        
        # alle gemessenen Bitstrings erhalten
        results = list(job.result().get_counts(copy_circuit).keys())
        # Stringformat in Int-array umwandeln
        results = list(map(lambda x: [int(c) for c in x], results))

        # Herausfiltern derer Bitstrings, welche das erlaubte Gesamtgewicht überschreiten
        results = list(filter(lambda x: (np.array(x) * np.array(self.weights)).sum() <= self.max_weight, results))

        # Resultierende Bitstrings in Werte ummünzen
        results = list(map(lambda x: (x, (np.array(x) * np.array(self.profits)).sum()), results))

        # Maximalen Wert mit zugehörigem Bitstring ausgeben
        return max(results, key=(lambda x: x[1])) if len(results) > 0 else []

In [195]:
class GridSearcher(object):
    
    def __init__(self, profits, weights, thetas, N_beta, N_gamma, p, solving_type, init_type, k=5):
        self.args = [profits, weights, thetas]
        self.N_beta = N_beta
        self.N_gamma = N_gamma
        self.p = p
        self.solver = Solver(*self.args + [solving_type, init_type, k])

        def generating_func(old_list, list, p):
            if p == 1:
                return old_list
            return [[e] + o_e for o_e in generating_func(old_list, list, p-1) for e in list]
        
        beta_starting_list = [np.pi * i / N_beta for i in range(N_beta)]
        self.betas = generating_func([[b] for b in beta_starting_list], beta_starting_list, self.p)
        gamma_starting_list = [2 * np.pi * i / N_gamma for i in range(N_gamma)]
        self.gammas = generating_func([[g] for g in gamma_starting_list], gamma_starting_list, self.p)


    def search_results(self):
        all_params = [b + g for b in self.betas for g in self.gammas]
        res = []
        for p in all_params:
            self.solver.set_params(p)
            res.append((p, self.solver.process_outcome()))
        return res

In [198]:
searcher = GridSearcher(profits=PROFITS, weights=(WEIGHTS, MAX_WEIGHT), thetas=THETA_S, N_beta=10, N_gamma=10, p=3, solving_type=SOLVING_TYPE, init_type=INIT_TYPE)

In [199]:
searcher.search_results()